<style>
    .title_div {
    padding: 60px;
    text-align: center;
    background: #ffffff;
    }
    #title {
        color: #f2a900;
        font-size: 80px;
    }
    #subtitle {
        color: #4d4d4e;
        font-size: 20px;
    }
</style>

<div class="title_div">
  <h1 id="title">SHA-256</h1>
  <p id="subtitle">Secure Hasing Algorithm 256 bits</p>
</div>

<div style="color:red; text-align: center; padding: 60px">
⚠ The following examples are just to see how SHA-256 work; no recommended to use this algorithms for hashing. ⚠
</div>


## TOC:
* [First look](#First-Look)
* [Atomic operations](#Atomic-Operations)
    1. [SHR](#SHR)
    2. [ROTR](#ROTR)
    3. [XOR](#XOR)
    4. [SUM](#SUM)
* [Coumpound operations](#Compound-Operations)
    1. [$\sigma0$](#sigma0)
    2. [$\sigma1$](#sigma1)
    3. [$\Sigma0$](#Sigma0)
    4. [$\Sigma1$](#Sigma1)
    5. [Choice](#Choice)
    6. [Majority](#Majority)
* [Constant K](#Constant-k)
* [Hashing](#Hashing)
    * [Encode](#Encode)
    * [Padding](#Padding)
    * [Message Schedule](#Message-Schedule)
        * [Block Iterator](#Block-Iterator)
        * [Word Iterator](#Word-Iterator)
        * [Initializing message schedule](#Initializing-message-schedule)
        * [Extending message shcedule](#Extending-message-schedule)
    * [Compression](#Compression)
        * [Initializing stage registers](#Initializing-stage-regs)
        * [Block Compression](#Block-Compression)
        * [Sum UP](#Sum-UP)
* [Checking the result](#Checking-Result)
* [Consolidating operations](#Consolidating-Operations)
* [Testing](#Testing)
    * [Simple Test](#Simple-Test)
    * [Multiblock Test](#Multiblock-Test)
    * [Unittest testing](#Unitest-Testing)

In [ ]:
# Needed modules for running all notebook cells
from IPython.display import clear_output
from bitarray import bitarray, util
from hashlib import sha256, sha512, new
from time import sleep
from math import sqrt, floor

# tests
import unittest

<a ID="First-Look"></a>
## First look

Using the SHA-256 function incuded in the hashlib module as reference.

In [ ]:
m = sha256()
print(f"Hash type:\t{m.name}")
print(f"Void digest:\t{m.hexdigest()}")
print(f"Digest size:\t{m.digest_size}")

string = "Hello World!"
m.update(string.encode())
print(f"\nString:\t{string}")
print(f"Digest:\t{m.hexdigest()}")

del m, string

<a ID="Atomic-Operations"></a>
## Atomic bit level operations

1. [SHR](#SHR)
2. [ROTR](#ROTR)
3. [XOR](#XOR)
4. [SUM](#SUM)

<a ID="SHR"></a>
### 1. SHR
Shift right, left bits populated with '0'.

In [ ]:
def shr(bits, shifts):
    for _ in range(shifts):
        bits.pop()
        bits.insert(0, 0)
    return bits

a = bitarray(endian='big')
a.frombytes(b'\xff\x00\xff\x00')

print("\n{:>8} {:>32}".format("A:", a.to01()))
for i in range(32):
    print("\r{:>8} {:>32}".format("SHR "+str(i+1)+":", shr(a,1).to01()), end="")
    sleep(0.2)
del a, i

<a ID="ROTR"></a>
### 2. ROTR
Rotation right, dropping bits over right appear on the left hand side.

In [ ]:
def rotr(bits, shifts):
    for _ in range(shifts):
        bits.insert(0,bits.pop())
    return bits

a = bitarray(endian='big')
a.frombytes(b'\xff\x00\xff\x00')

print("\n{:>8} {:>32}".format("A:", a.to01()))
for i in range(32):
    print("\r{:>8} {:>32}".format("ROTR "+str(i+1)+":", rotr(a,1).to01()), end="")
    sleep(0.2)
del a, i

<a ID="XOR"></a>
### 3. XOR
Exclusive OR



| A | B | XOR |
|---|---|-----|
| 0 | 0 | 0   |
| 0 | 1 | 1   |
| 1 | 0 | 1   |
| 1 | 1 | 0   |   



In [ ]:
def xor(a, b):
    return a^b

a = bitarray(endian='big')
b = bitarray(endian='big')
a.frombytes(b'\xff\x00\xff\x00')
b.frombytes(b'\x00\xff\xff\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
print("{:>8} {:>32} XOR".format("B:", b.to01()))
print("{:>8} {:>32}".format("", "-"*32))
for i in range(32):
    print("\r{:>8} {:>32}".format("", xor(a,b).to01()[32-1-i:]), end="")
    sleep(0.2)
del a, b, i


<a ID="SUM"></a>
### 4. SUM
Binary addition <br>
Mod 32, ignoring the carry out that will overflow the 32 bit.

In [ ]:
def add(a, b):
    result = bitarray(endian='big')
    carry = 0
    for i in range(32-1, -1, -1):
        result.insert(0,(a[i]^b[i]^carry))
        carry = 1 if (a[i] + b[i] + carry) > 1 else 0
    return result


a = bitarray(endian='big')
b = bitarray(endian='big')
a.frombytes(b'\x80\x30\xff\xff')
b.frombytes(b'\x80\x50\x00\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
print("{:>8} {:>32} +".format("B:", b.to01()))
print("{:>8} {:>32}".format("", "-"*32))
out = add(a,b)
for i in range(32):
    print("\r{:>8} {:>32}".format("", out.to01()[32-1-i:]), end="")
    sleep(0.2)

del a, b, i, out

***
<a ID="Compound-Operations"></a>
## Compound operations

1. [$\sigma0$](#sigma0)
2. [$\sigma1$](#sigma1)
3. [$\Sigma0$](#Sigma0)
4. [$\Sigma1$](#Sigma1)
5. [Choice](#Choice)
6. [Majority](#Majority)


<a ID="sigma0"></a>
### 1. $\sigma0$

In [ ]:
def sigma0(a, b, c):
    rotr(a, 7)
    rotr(b, 18)
    shr(c, 3)
    return xor(xor(a,b),c)

origin = bitarray(endian='big')
origin.frombytes(b'\x00\x00\x3f\xff')

a = origin.copy()
b = origin.copy()
c = origin.copy()

result = sigma0(a, b, c)

print("{:>8} {:>32}".format("Origin:", origin.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("ROTR  7:", a.to01()))
print("{:>8} {:>32} XOR".format("ROTR 18:", b.to01()))
print("{:>8} {:>32} XOR".format("SHR  7:", c.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("\u03C30:", result.to01()))

del a, b, c, result, origin



<a ID="sigma1"></a>
### 1. $\sigma1$

In [ ]:
def sigma1(a, b, c):
    rotr(a, 17)
    rotr(b, 19)
    shr(c, 10)
    return xor(xor(a,b),c)

origin = bitarray(endian='big')
origin.frombytes(b'\x00\x00\x3f\xff')

a = origin.copy()
b = origin.copy()
c = origin.copy()

result = sigma1(a, b, c)

print("{:>8} {:>32}".format("Origin:", origin.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("ROTR 17:", a.to01()))
print("{:>8} {:>32} XOR".format("ROTR 19:", b.to01()))
print("{:>8} {:>32} XOR".format("SHR 10:", c.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("\u03C31:", result.to01()))

del a, b, c, result, origin


<a ID="Sigma0"></a>
### 3. $\Sigma0$

In [ ]:
def Sigma0(a, b, c):
    rotr(a, 2)
    rotr(b, 13)
    rotr(c, 22)
    return xor(xor(a,b),c)

origin = bitarray(endian='big')
origin.frombytes(b'\x00\x00\x3f\xff')

a = origin.copy()
b = origin.copy()
c = origin.copy()

result = Sigma0(a, b, c)

print("{:>8} {:>32}".format("Origin:", origin.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("ROTR  2:", a.to01()))
print("{:>8} {:>32} XOR".format("ROTR 13:", b.to01()))
print("{:>8} {:>32} XOR".format("ROTR 22:", c.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("\u03A30:", result.to01()))

del a, b, c, result, origin


<a ID="Sigma1"></a>
### 4. $\Sigma1$

In [ ]:
def Sigma1(a, b, c):
    rotr(a, 6)
    rotr(b, 11)
    rotr(c, 25)
    return xor(xor(a,b),c)

origin = bitarray(endian='big')
origin.frombytes(b'\x00\x00\x3f\xff')

a = origin.copy()
b = origin.copy()
c = origin.copy()

result = Sigma1(a, b, c)

print("{:>8} {:>32}".format("Origin:", origin.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("ROTR  6:", a.to01()))
print("{:>8} {:>32} XOR".format("ROTR 11:", b.to01()))
print("{:>8} {:>32} XOR".format("ROTR 25:", c.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("\u03A31:", result.to01()))

del a, b, c, result, origin

<a ID="Choice"></a>
### 5. Choice

Choose the bit at possition $X$ from 'a' in case choice bit '0' or from 'b' in case choice bit '1'.

In [ ]:
def choice(a, b, c):
    result = a.copy()
    for i in range(32):
        if a[i] == 1:
            result[i] = b[i]
        else:
            result[i] = c[i]
    return result

a = bitarray(endian='big')
b = bitarray(endian='big')
c = bitarray(endian='big')

a.frombytes(b'\x00\xff\x00\xff')
b.frombytes(b'\x00\x00\xff\xff')
c.frombytes(b'\xff\xff\x00\x00')

result = choice(a, b, c)

for i in range(32):

    print(" "*(41-(i+1))+"\u21D3")
    print("{:>8} {:>32}".format("A:", a.to01()))
    print("{:>8} {:>32} {}".format("B:", b.to01(), "" if a[i]==1 else "\u21E6"))
    print("{:>8} {:>32} {}".format("C:", c.to01(), "" if a[i]==0 else "\u21E6"))
    print("{:>8} {:>32}".format("", "-"*32))
    print("{:>8} {:>32}".format("OUT:", result.to01()[32-(i+1):]))
    
    sleep(0.2)
    clear_output(wait=True)
    

del a, b, c, result

<a ID="Majority"></a>
### 6. Majority

In [ ]:
def majority(a, b, c):
    result = bitarray("0"*32)
    for i in range(32):
        result[i] = 1 if a[i]+b[i]+c[i] > 1 else 0
    return result

a = bitarray(endian='big')
b = bitarray(endian='big')
c = bitarray(endian='big')

a.frombytes(b'\x00\xff\x00\xff')
b.frombytes(b'\x00\x00\xff\xff')
c.frombytes(b'\xff\xff\x00\x00')

result = majority(a, b, c)

for i in range(32):

    print(" "*(41-(i+1))+"\u21D3")
    print("{:>8} {:>32} {}".format("A:", a.to01(), " "+str(a[32-(i+1)])))
    print("{:>8} {:>32} {}".format("B:", b.to01(), " "+str(b[32-(i+1)])))
    print("{:>8} {:>32} {}".format("C:", c.to01(), " "+str(c[32-(i+1)])))
    print("{:>8} {:>32}".format("", "-"*32))
    print("{:>8} {:>32}".format("OUT:", result.to01()[32-(i+1):]))
    
    sleep(0.2)
    clear_output(wait=True)
    

del a, b, c, result

***
<a Id="Constant-K"></a>
## Constant K
List of length 64 containing 32 bits from the decimal part of the cubic root of the 64 first prime numbers.

In [ ]:
first64primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89,
 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197,
 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]

K = [] # The list is indexed from 0 even tho is printed starting from 1.

def frac2bin(frac):
    x = floor(frac*(2**32))
    bin32 = ""
    
    for _ in range(32):
        bin32 += str(x % 2)
        x = x // 2

    return bin32[::-1]
    


for j, i in enumerate(first64primes):
    sqrt3 = i**(1./3.)
    deciaml_part = float(str(sqrt3)[1:])
    K.append(bitarray(frac2bin(deciaml_part)))

    print("{:>4} \u21FE {:>4} \u21FE {:>18} \u21FE {:>18} \u21FE {:>32}  ".format(j+1, "\u221B"+str(i), sqrt3, deciaml_part,K[j].to01()))

del first64primes, deciaml_part, sqrt3, i, j



***
<a ID="Hashing"></a>
## Hashing

1. [Encode](#Encode)
2. [Padding](#Padding)
3. [Message Schedule](#Message-Schedule)
    1. [Block Iterator](#Block-Iterator)
    2. [Word Iterator](#Word-Iterator)
    3. [Initializing message schedule](#Initializing-message-schedule)
    4. [Extending message shcedule](#Extending-message-schedule)
4. [Compression](#Compression)
    1. [Initializing stage registers](#Initializing-stage-regs)
    1. [Block Compression](#Block-Compression)
    3. [Sum UP](#Sum-UP)



<a ID="Encode"></a>
### Encode
Always use the same encode to have a solid and consistent output; in the case of SHA256 we use ASCII (American Standard Code II)

In [ ]:
string = "Hello World!"
message = bitarray(endian='big')
message.frombytes(string.encode('ascii')) # passing it to binary with a ASCII encoding

print("Message:")
print("\t{} \"{}\"".format("Input:", string))
print("\tEncoding: ASCII")
print("\t{} {}".format("Message:", message.to01()))



<a ID="Padding"></a>
### Padding
Hashing algorithms work in badges or blocks. Padding adds bits to make the message length a multiple of the block size required.

In [ ]:
print(f"SHA-256 block size: {sha256().block_size*8} bits")

Pdding structure:
* Padding flag bit, added right after the message.
* Cero padding, ceros added to the message in order to make its length a multiple of the block.
* Length on the message encoded in the last 64 bits.

In [ ]:
def padding(message):

    p_message = message.to01()
    blocks = len(message) // 512 +1 # Number of block the message is gonna take

    if (len(message) % 512 >= (512 - 65)):
        blocks += 1

    p_message += '1' # Padding flag

    # Same result as the commented loop
    p_message += '0'*(blocks*512 - 64 - len(p_message))
    #for _ in range(len(p_message), blocks*512 - 64):
    #    p_message += '0'

    # Last 64 bits with message length
    bin_len_message = bin(len(message))[2:]
    len_bin_len_message = len(bin_len_message)
    p_message += '0'*(64-len_bin_len_message) + bin_len_message

    return bitarray(p_message)



p_message = padding(message)

print(f"Padding message size: {len(p_message)} bits")
print("Message:")

colors = {'Message':'\033[95m', 'Padding flag':'\033[91m', 'Padding':'\033[93m', 'Length':'\033[96m', 'Message Length':'\033[94m'}

for i, bit in enumerate(p_message.to01()):
    if i >= len(p_message) - len(bin(len(message))[2:]):
        color = colors['Message Length']
    elif i >= len(p_message) - 64:
        color = colors['Length']
    elif i >= len(message) + 1:
        color = colors['Padding']
    elif i >= len(message):
        color = colors['Padding flag']
    else:
        color = colors['Message']
    print(color+bit, end="")

print(f"\n{colors['Message']}\u2589 {'Message'}")
print(f"{colors['Padding flag']}\u2589 {'Padding flag'}")
print(f"{colors['Padding']}\u2589 {'Padding'}")
print(f"{colors['Length']}\u2589 {'Reserved bits for mesage length'}")
print(f"{colors['Message Length']}\u2589 {'Message length'}")

del color, colors, bit, i

<a ID='Message-Schedule'></a>
### Message Schedule

<a ID="Block-Iterator"></a>
#### Block Iterator
Iterates over the chunks "blocks" of data of the padding message.

In [ ]:
def blck_iter(message):
    N_blck = len(message) // 512
    for blck in range(N_blck):
        yield message[blck*512:(blck+1)*512]

<a ID="Word-Iterator"></a>
#### Word Iterator
Iterates over words (32 bit chuncks) of the block.

In [ ]:
def word_iter(block):
    N_words = len(block) // 32
    for word in range(N_words):
        yield block[word*32:(word+1)*32]

<a ID='Initializing-message-schedule'></a>
#### Initializing message schedule
Creates a list of 16 words of 32 bits each, from the block data.

In [ ]:

def init_message_schedule(message):

    words = []

    for i in blck_iter(message):
        words = []
        for j in word_iter(i):
            words.append(j)
        yield words

word_list = next(init_message_schedule(p_message))

for i, w in enumerate(word_list):
    print("{:>4}: {}".format("W"+str(i+1), w.to01()))

del i, w

<a ID='Extending-message-schedule'></a>
#### Extending message schedule

Compute the following formula for the remaining 48 words, for a total of 64 (48 + 16 = 64). <br></br>
$Wx = σ1(W(x-2)) + W(x-7) + σ0(W(x-15)) + W(x-16)$

In [ ]:
def extend_message_schedule(words):
    for i in range(64-16):
        N = len(words)
        a = words[N-16].copy()
        b = sigma0(*[words[N-15].copy() for _ in range(3)]) # Weird unpaking to match args
        c = words[N-7].copy()
        d = sigma1(*[words[N-2].copy() for _ in range(3)]) # Weird unpaking to match args
        words.append(add(add(a, b), add(c, d)))
    return words
    
    
if(len(word_list) < 64):
    # To refresh extended word list you must first rerun the aboce cell
    word_list = extend_message_schedule(word_list.copy())


for i in range(16, len(word_list)-15):

    for j in range(i, i + 16):
        if (j == i):
            right_side = f"\u2014"*5+f"\u2192 {word_list[j].to01()}"
        elif( j == i + 1):
            right_side = f"\u2014 \u03C30 \u2192 {sigma0(*[word_list[j].copy() for _ in range(3)]).to01()}"
        elif( j == i + 8):
            right_side = f"\u2014"*5+f"\u2192 {word_list[j].to01()}"
        elif( j == i + 13):
            right_side = f"\u2014 \u03C31 \u2192 {sigma1(*[word_list[j].copy() for _ in range(3)]).to01()}"
        elif( j == i + 15):
            right_side = "\u2190"+"\u2014"*5+f" \u03C31(W{i + 14}) + W{i + 9} + \u03C30(W{i + 2}) + W{i + 1}"
        else:
            right_side = ""
        print("{:>4}: {:>32} {}".format("W"+str(j+1), word_list[j].to01(), right_side))

    sleep(0.3)
    clear_output(wait=True)

del i, j, right_side

<a ID='Compression'></a>
### Compression
<a ID='Initializing-stage-regs'></a>
#### Initializing stage registers
Initializing stage registers with the 32 first bits of the fractional part of the 8 first primes square roots.

In [ ]:
def init_stage_registers():
    regs = [2, 3, 5, 7, 11, 13, 17, 19]
    for i in range(len(regs)):
        aux = bin(int((sqrt(regs[i])%1)*2**32))[2:]
        regs[i] = bitarray("0"*(32-len(aux))+aux)
    return regs

stage_reg = init_stage_registers()

for i, p in zip([2, 3, 5, 7, 11, 13, 17, 19], stage_reg):
    print("{:>4} \u21FE {:>4} \u21FE {:>18} \u21FE {:>20} \u21FE {:>32}  ".format(i, "\u221A"+str(i), sqrt(i), sqrt(i)%1, p.to01()))
    #print("{:>2} {}".format(i, p.to01()))

del i, p

<a ID="Block-Compression"></a>
#### Block compression

For the compression step we take our stage or working register we had previusly initialize, the first word from our schedule, the first K constant, and solve the following operations:

$T1 = Σ1(e) + Choice(e, f, g) + h + Ki + Wi$<br>
$T2 = Σ0(a) + Majority(a, b, c)$
<div class="alert alert-block alert-info">
<b>Tip:</b> 'i' is the index in our word schelu array and K array.
</div>

Then we shift each of the 8 registers moving the last one out of the array and populating the first one with ( $T1 + T2$ ). We also update 5th register by adding $T1$ to it and repeat this two steps for each Word and (K) constant pair.

In [ ]:
def block_compress(stage_reg, word_list, K, animation=False):

    working_stage_reg = stage_reg

    temp1 = bitarray(endian='big')
    temp2 = bitarray(endian='big')

    for i in range(64):
        temp1 = add( add( add( Sigma1(*[working_stage_reg[4].copy() for _ in range(3)]), choice(working_stage_reg[4].copy(), working_stage_reg[5].copy(), working_stage_reg[6].copy())), add( working_stage_reg[7].copy(), K[i])), word_list[i])
        temp2 = add(Sigma0(*[working_stage_reg[0].copy() for _ in range(3)]), majority(working_stage_reg[0].copy(), working_stage_reg[1].copy(), working_stage_reg[2].copy()))
        top = working_stage_reg[0].copy()
        working_stage_reg.insert(0, add(temp1, temp2))
        trash = working_stage_reg.pop()
        working_stage_reg[4] = add(working_stage_reg[4], temp1)

        if animation:
            for scene in range(3):
                print("W{}: {}".format(i+1, word_list[i].to01()))
                print("K{}: {}\n".format(i+1, K[i].to01()))

                print("T1: {:>32} = {}".format(temp1.to01(), "\u03A31(e) + Choice(e, f, g) + h + K"+str(i+1)+" + W"+str(i+1)))
                print("T2: {:>32} = {}\n".format(temp2.to01(), "\u03A30(a) + Majority(a, b, c)"))

                if(scene == 0):
                    right_side = "\u2193"
                    print("{:>4} = {:>32} {}".format(chr(97), top.to01(), right_side))
                    for j, reg in enumerate(working_stage_reg[:-1]):
                        print("{:>4} = {:>32} {}".format(chr(j+97+1), reg.to01(), right_side))
                    print("")
                elif(scene == 1):
                    for j, reg in enumerate(working_stage_reg):
                        if( j == 0):
                            reg = "..."
                            right_side = ""
                            print("{:>4} = {:<32} {}".format(chr(j+97), reg, right_side))
                        else:
                            right_side = ""
                            print("{:>4} = {:>32} {}".format(chr(j+97), reg.to01(), right_side))
                    print("{:>4} {:>32} {}".format("\U0001F6AE", trash.to01(), right_side))
                elif(scene == 2):
                    for j, reg in enumerate(working_stage_reg):
                        if( j == 0):
                            right_side = "\u2190"+"\u2014 T2 + T1"
                        elif( j == 4):
                            right_side = "\u2190"+"\u2014 e + T1"
                        else:
                            right_side = ""
                        print("{:>4} = {:>32} {}".format(chr(j+97), reg.to01(), right_side))
                    print("")

                if i < 12:
                    sleep(0.5)
                else:
                    sleep(0.05)
                clear_output(wait=True)
    
    return working_stage_reg

stage_reg = block_compress(stage_reg.copy(), word_list, K, True)


<a ID="Sum-UP"></a>
### Sum UP

As the final step we add the stage or working registers with the values of the initial registers, and encode this to hexadecimal. Then concatenate the hexadecimal values from first to last on a sigle string, and we have our hash digest.

In [ ]:

def sum_up(stage_reg, animation = False):
    result = []
    hexa = []
    if animation:
        print("{:8}   {:32}   {:32}   {:32}".format("HEX".center(8), "Results".center(32), "Initial reg values".center(32), "Final compression values".center(32)))
        print("-"*113)
    for init_reg, temp_reg in zip(init_stage_registers(), stage_reg):
        result.append(add(init_reg, temp_reg))
        hexa.append(util.ba2hex(result[-1]))
        if animation:
            print("{} \u2190 {} = {} + {}".format(hexa[-1], result[-1].to01(), init_reg.to01(), temp_reg.to01()))
    return "".join(hexa)
        

digest = sum_up(stage_reg, True)

print("\n\t\u250F"+"\u2501"*74+"\u2513")
print("\t\u2503 Digest: "+digest+" \u2503")
print("\t\u2517"+"\u2501"*74+"\u251B")

***
<a ID="Checking-Result"></a>
## Checking the result

Checking the result of our hashing fuction against the SHA-256 build in hashlib module.

In [ ]:
def compare_digests(digest1, digest2):
    printout1 = ""
    printout2 = ""
    for i, j in zip(digest1, digest2):
        if i != j:
            color = "\033[91m"
        else:
            color = "\033[92m"
        printout1 += color+i
        printout2 += color+j
    check = "\u2705" if digest1 == digest2 else "\u274C"
    
    print(printout1+" "+printout2+" "+check+"\033[0m")

print(f"{'Type'.center(9)} {'Our digest'.center(64)} {'Hashlib digest'.center(64)}\n"+"-"*145)
print(f"  \"Correct\"   ", end="")
compare_digests(digest, sha256(string.encode('ascii')).hexdigest())
print("  \"Wrong\"     ", end="")
compare_digests(digest, sha256(b"Hello Kitty!").hexdigest()) # Wrong on purpose, to see different output


<a ID="Consolidating-Operations"></a>
## Consolidating operations

All the diferent parts and steps putted tougether in a single fucntion that can handle more than one block hashes.

In [ ]:
def diy_sha256(data):
    message = bitarray(endian='big')
    message.frombytes(data)
    message = padding(message)
    #print(message)
    virgin_stage_registers = init_stage_registers()
    m_schedule = init_message_schedule(message)
    working_stage_registers = init_stage_registers()#virgin_stage_registers
    
    #print(virgin_stage_registers)
    for i, word_batch in enumerate(m_schedule):
        
        
        #print(working_stage_registers)
        word_batch = extend_message_schedule(word_batch.copy())
        working_stage_registers = block_compress(working_stage_registers, word_batch, K)

        for n in range(8):
            working_stage_registers[n] = add(virgin_stage_registers[n], working_stage_registers[n])

        virgin_stage_registers = working_stage_registers.copy()
    
    hex_regs = []
    for i in working_stage_registers:
        hex_regs.append(util.ba2hex(i))

    #print(hex_regs)
    
    return "".join(hex_regs) #sum_up(working_stage_registers)
    


<a ID="Testing"></a>
## Testing

<a ID="Simple-test"></a>
#### Simple test

Uses a word list to check for a couple of diferent inputs.

<div class="alert alert-block alert-warning">
<b>Tip:</b> The file "BIP-0039_english.txt" must be in the same directory as the notebook.
</div>

In [ ]:
print(f"{'   N'}"+" "*7+f"{'Input'} {'Our digest'.center(64)} {'Hashlib digest'.center(64)}\n"+"-"*150)
with open("BIP-0039_english.txt", 'r') as file:
    for i, item in enumerate(file):
        if i % 85 == 0:
            print("{:>4} {:>12} ".format(i,item.rstrip()), end="")
            binary_item = item.rstrip().encode('ascii')
            compare_digests(diy_sha256(binary_item), sha256(binary_item).hexdigest())

<a ID="Multiblock-Test"></a>
#### Multiblock test

The following code increments the size of a string in each of its iterations to proof the algorith with different block sizes.

In [ ]:

print("{:>4}  {:>4} {:>64} {:>64}\n".format("Bits", "Blcks", 'Our digest'.center(64), 'Hashlib digest'.center(64))+"-"*140)
for i in range(10):
    long_string = "SHA-256"*i*10
    print("{:>4} {:>4} ".format(len(long_string)*8, len(long_string) // sha256().block_size), end=" ")
    bin_long_string = long_string.encode('ascii')
    compare_digests(diy_sha256(bin_long_string), sha256(bin_long_string).hexdigest())


<a class="anchor" id="Unitest-Testing"></a>
#### Unittest testing

In [ ]:
class TestSHA256(unittest.TestCase):
    def test_sigleblock(self):
        with open("BIP-0039_english.txt", 'r') as file:
            for i, item in enumerate(file):
                binary_item = item.rstrip().encode('ascii')
                diy_digest = diy_sha256(binary_item)
                hashlib_digest = sha256(binary_item).hexdigest()
                self.assertEqual(diy_digest, hashlib_digest)

    def test_multiblok(self):
        for i in range(200, 500):
                binary_item = ('0'*i).encode('ascii')
                diy_digest = diy_sha256(binary_item)
                hashlib_digest = sha256(binary_item).hexdigest()
                self.assertEqual(diy_digest, hashlib_digest)

    def test_empty(self):
        binary_item = "".encode('ascii')
        diy_digest = diy_sha256(binary_item)
        hashlib_digest = sha256(binary_item).hexdigest()
        self.assertEqual(diy_digest, hashlib_digest)

    def test_loremipsum(self):
        text = """Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque ex nulla, tincidunt sed pretium vitae, sagittis nec urna.
        Donec tempus scelerisque semper. Aliquam ut vestibulum ex, a varius tortor. Suspendisse eget metus condimentum, tincidunt nulla quis, eleifend nunc.
        Vivamus sed volutpat eros. Vestibulum malesuada ac orci in fermentum. Aenean fringilla, elit in efficitur facilisis, arcu dolor convallis arcu, sed finibus leo orci id turpis.
        Aliquam erat volutpat. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia curae; Duis blandit congue massa. Aenean scelerisque turpis id elit aliquet efficitur.
        Nam dignissim semper ultricies. Aliquam sed nunc in ligula finibus egestas eu eu mi. Donec pharetra nunc id turpis iaculis sagittis. Nulla in eleifend arcu.
        Donec porta elit at cursus elementum. Nulla consectetur varius dui ac congue. Suspendisse at nisl ut augue placerat cursus at at mauris. Fusce ultricies dignissim velit, eu sodales ex venenatis augue."""
        binary_item = text.encode('ascii')
        diy_digest = diy_sha256(binary_item)
        hashlib_digest = sha256(binary_item).hexdigest()
        self.assertEqual(diy_digest, hashlib_digest)
    
    def test_failbyonebyte(self):
        binary_ascii = b'Hello World'
        binary_utf = b'Hello world'
        ascii_digest = diy_sha256(binary_ascii)
        utf_digest = diy_sha256(binary_utf)
        self.assertNotEqual(ascii_digest, utf_digest)



unittest.main(argv=[''], verbosity=2, exit=False)